<a href="https://colab.research.google.com/github/linqus/nlp-huggingface/blob/main/notebooks/unit3/nlp_3_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets accelerate

     ---------------------------------------- 0.0/129.4 kB ? eta -:--:--
     --- ------------------------------------ 10.2/129.4 kB ? eta -:--:--
     --- ------------------------------------ 10.2/129.4 kB ? eta -:--:--
     -------- ---------------------------- 30.7/129.4 kB 217.9 kB/s eta 0:00:01
     ----------------- ------------------- 61.4/129.4 kB 363.1 kB/s eta 0:00:01
     ---------------------------------- - 122.9/129.4 kB 602.4 kB/s eta 0:00:01
     ------------------------------------ 129.4/129.4 kB 586.8 kB/s eta 0:00:00
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 42.0/42.0 kB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB 3.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ---

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

c:\Users\Admin\anaconda3\envs\cuda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 1725/1725 [00:00<00:00, 105718.67 examples/s]
tokenizer_config.json: 100%|██████████| 28.0/28.0 [00:00<?, ?B/s]
c:\Users\Admin\anaconda3\envs\cuda\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limita

In [3]:
tokenized_datasets['train'][:4].keys()

dict_keys(['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'])

In [4]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
#tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format('torch')




In [5]:
tokenized_datasets['train'].column_names

['label', 'input_ids', 'token_type_ids', 'attention_mask']

In [6]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_datasets['train'], shuffle=True, batch_size=8, collate_fn=data_collator)
eval_dataloader = DataLoader(tokenized_datasets['validation'], batch_size=8, collate_fn=data_collator)



In [7]:
len ([len(batch['input_ids'][0]) for batch in train_dataloader])



459

In [8]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.safetensors: 100%|██████████| 440M/440M [00:06<00:00, 69.7MB/s] 
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

c:\Users\Admin\anaconda3\envs\cuda\Lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [10]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)

scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

print(num_training_steps)

1377


In [11]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [12]:
from tqdm.auto import tqdm

progress = tqdm(range(num_training_steps))

model.train()
#i = 0
for epoch in range(num_epochs):
  for batch in train_dataloader:

    #i = i + 1
    #print(epoch, i, f"z {num_training_steps}")

    batch = {k: v.to(device) for k, v in batch.items()}

    outputs = model(**batch)
    loss = outputs.loss
    loss.backward()

    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

    progress.update()




100%|██████████| 1377/1377 [02:02<00:00, 11.74it/s]

In [13]:
!pip install evaluate

100%|██████████| 1377/1377 [02:16<00:00, 11.74it/s]

   ---------------------------------------- 0.0/84.1 kB ? eta -:--:--
   -------------- ------------------------- 30.7/84.1 kB ? eta -:--:--
   ---------------------------------------- 84.1/84.1 kB 1.6 MB/s eta 0:00:00


In [16]:
!pip install scikit-learn scipy

     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.4 kB ? eta -:--:--
     -------------------------------- ----- 51.2/60.4 kB 650.2 kB/s eta 0:00:01
     -------------------------------------- 60.4/60.4 kB 796.9 kB/s eta 0:00:00
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/10.6 MB 2.4 MB/s eta 0:00:05
   - -------------------------------------- 0.4/10.6 MB 5.3 MB/s eta 0:00:02
   -- ------------------------------------- 0.8/10.6 MB 6.1 MB/s eta 0:00:02
   ----- ---------------------------------- 1.5/10.6 MB 8.4 MB/s eta 0:00:02
   -------- ------------------------------- 2.3/10.6 MB 10.4 MB/s eta 0:00:01
   ---------- ----------------------------- 2.9/10.6 MB 10.9 MB/s eta 0:00:01
   ------------- -------------------------- 3.7/10.6 MB 11.6 MB/s eta 0:00:01
   ----------------- ---------------------- 4.6/10.6 MB 12.8 MB/s eta 0:00:01
   --

In [17]:
import evaluate


metrics = evaluate.load("glue", "mrpc")
model.eval()

for batch in eval_dataloader:
  batch = {k: v.to(device) for k, v in batch.items()}
  with torch.no_grad():
    outputs = model(**batch)

  logits = outputs.logits
  preds = torch.argmax(logits, axis=-1)
  metrics.add_batch(predictions=preds,references=batch["labels"])

metrics.compute()

{'accuracy': 0.8602941176470589, 'f1': 0.9028960817717206}

In [18]:
batch.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [19]:
outputs

SequenceClassifierOutput(loss=tensor(0.0351, device='cuda:0'), logits=tensor([[ 1.1355, -1.3939],
        [-2.9975,  3.5911],
        [-1.6916,  2.1205],
        [ 1.9230, -2.4123],
        [-0.7269,  1.0592],
        [-3.1177,  3.6123],
        [ 2.3828, -2.7045],
        [-2.3197,  2.8463]], device='cuda:0'), hidden_states=None, attentions=None)

From scratch

In [20]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])


Map: 100%|██████████| 408/408 [00:00<00:00, 4791.30 examples/s]


In [21]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_datasets['train'], shuffle=True, batch_size=8, collate_fn=data_collator)
eval_dataloader = DataLoader(tokenized_datasets['validation'], batch_size=8, collate_fn=data_collator)

In [30]:
!pip install tqdm

In [37]:
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler
import evaluate, torch
from tqdm.auto import tqdm
from accelerate import Accelerator # accelerate

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)

#device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
#model.to(device)

accelerator = Accelerator()
train_dataloader, eval_dataloader, model, optimizer  = accelerator.prepare(train_dataloader, eval_dataloader, model, optimizer)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

progress_bar = tqdm(range(num_training_steps), position=0, leave=True)

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        #batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        #loss.backward()
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

model.eval()

metrics = evaluate.load("glue", "mrpc")

for batch in eval_dataloader:
  batch = {k: v.to(device) for k, v in batch.items()}
  with torch.no_grad():
    outputs = model(**batch)

  logits = outputs.logits
  preds = torch.argmax(logits, axis=-1)
  metrics.add_batch(predictions=preds,references=batch["labels"])

metrics.compute()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|█████████▉| 1376/1377 [02:12<00:00, 10.84it/s]

{'accuracy': 0.8774509803921569, 'f1': 0.9143835616438356}

100%|██████████| 1377/1377 [02:22<00:00, 10.84it/s]